In [7]:
# aggregate_results.ipynb
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Set working directory
base_dir = "./results"

# Get all datasets
datasets = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
print(f"Found {len(datasets)} datasets: {datasets}")

# Create result storage structure
all_results = []

# Iterate through all datasets and seeds
for dataset in datasets:
    dataset_dir = os.path.join(base_dir, dataset)
    seeds = [s for s in os.listdir(dataset_dir) if s.startswith('seed_')]
    
    print(f"Processing dataset {dataset}, {len(seeds)} seeds in total")
    
    # Collect results for all seeds in this dataset
    dataset_results = []
    
    for seed in seeds:
        result_file = os.path.join(dataset_dir, seed, "correlation_results.csv")
        
        # Check if file exists
        if not os.path.exists(result_file):
            print(f"Warning: File not found {result_file}")
            continue
            
        # Read result file
        try:
            df = pd.read_csv(result_file)
            df['dataset'] = dataset
            df['seed'] = seed
            dataset_results.append(df)
        except Exception as e:
            print(f"Error reading {result_file}: {e}")
    
    # Merge all results for this dataset
    if dataset_results:
        dataset_df = pd.concat(dataset_results, ignore_index=True)
        all_results.append(dataset_df)

# Merge results from all datasets
if all_results:
    results_df = pd.concat(all_results, ignore_index=True)
    
    # Display dataset and seed coverage
    print("\nDataset and seed coverage:")
    coverage = results_df.groupby(['dataset', 'seed']).size().unstack(0)
    display(coverage)
    
    # Calculate mean for each dataset, method, metric
    print("\nCalculating averages grouped by dataset, method and metric:")
    avg_by_dataset = results_df.groupby(['dataset', 'split', 'method', 'metric'])['value'].agg(['mean', 'std']).reset_index()
    
    # First display results for each dataset
    for dataset in datasets:
        dataset_avg = avg_by_dataset[avg_by_dataset['dataset'] == dataset]
        if len(dataset_avg) > 0:
            print(f"\nDataset: {dataset}")
            # Pivot table for easier viewing
            pivot = dataset_avg.pivot_table(
                index=['split', 'method'], 
                columns='metric', 
                values='mean'
            )
            display(pivot)
    
    # Average across all datasets
    print("\nAverage results across all datasets:")
    overall_avg = results_df.groupby(['split', 'method', 'metric'])['value'].agg(['mean', 'std']).reset_index()
    pivot_overall = overall_avg.pivot_table(
        index=['split', 'method'], 
        columns='metric', 
        values='mean'
    )
    display(pivot_overall)
    
    # Save summary results
    avg_by_dataset.to_csv("aggregate_results_by_dataset.csv", index=False)
    overall_avg.to_csv("aggregate_results_overall.csv", index=False)
    
    print("\nResults saved to 'aggregate_results_by_dataset.csv' and 'aggregate_results_overall.csv'")
else:
    print("No result files found")

# Create a function to print the best methods
def print_best_methods(df, metrics=None, higher_better=None):
    if metrics is None:
        metrics = ['mae', 'mse']
    
    if higher_better is None:
        higher_better = {
            'mae': False,
            'mse': False
        }
    
    print("\nBest method for each metric:")
    for metric in metrics:
        metric_df = df[df['metric'] == metric]
        
        # Determine whether to take max or min based on the metric
        if higher_better.get(metric, True):
            best_idx = metric_df['mean'].idxmax()
        else:
            best_idx = metric_df['mean'].idxmin()
        
        best_row = metric_df.loc[best_idx]
        print(f"{metric}: {best_row['method']} (value: {best_row['mean']:.4f} ± {best_row['std']:.4f})")

# Print the best method for each metric
if 'overall_avg' in locals():
    print_best_methods(overall_avg)

Found 8 datasets: ['MiniBooNE', '2dplanes', 'digits', 'bbc-embeddings', 'nomao', 'electricity', 'election', 'fried']
Processing dataset MiniBooNE, 20 seeds in total
Processing dataset 2dplanes, 20 seeds in total
Processing dataset digits, 20 seeds in total
Processing dataset bbc-embeddings, 20 seeds in total
Processing dataset nomao, 20 seeds in total
Processing dataset electricity, 20 seeds in total
Processing dataset election, 20 seeds in total
Processing dataset fried, 20 seeds in total

Dataset and seed coverage:


dataset,2dplanes,MiniBooNE,bbc-embeddings,digits,election,electricity,fried,nomao
seed,,,,,,,,
seed_10,24,24,24,24,24,24,24,24
seed_100,24,24,24,24,24,24,24,24
seed_110,24,24,24,24,24,24,24,24
seed_120,24,24,24,24,24,24,24,24
seed_130,24,24,24,24,24,24,24,24
seed_140,24,24,24,24,24,24,24,24
seed_150,24,24,24,24,24,24,24,24
seed_160,24,24,24,24,24,24,24,24
seed_170,24,24,24,24,24,24,24,24



Calculating averages grouped by dataset, method and metric:

Dataset: MiniBooNE


metric                  mae       mse
split method                         
test  Banzhaf      0.317504  0.136565
      BetaShapley  1.281986  2.760100
      Bipartite    0.101351  0.018270
      Linear       0.265365  0.096057
      MLP          0.315962  0.132002
      Shapley      1.311503  2.888855
train Banzhaf      0.314128  0.130298
      BetaShapley  1.280374  2.800790
      Bipartite    0.104437  0.020153
      Linear       0.255039  0.090661
      MLP          0.323584  0.132988
      Shapley      1.308422  2.926480


Dataset: 2dplanes


...


Dataset: fried


metric                  mae       mse
split method                         
test  Banzhaf      0.312604  0.131974
      BetaShapley  1.270510  2.716354
      Bipartite    0.124394  0.025687
      Linear       0.260583  0.092242
      MLP          0.311154  0.127505
      Shapley      1.298453  2.838152
train Banzhaf      0.308200  0.124793
      BetaShapley  1.268388  2.753805
      Bipartite    0.127549  0.027603
      Linear       0.249103  0.086042
      MLP          0.317377  0.127352
      Shapley      1.295133  2.873245


Average results across all datasets:


metric                  mae       mse
split method                         
test  Banzhaf      0.302801  0.127894
      BetaShapley  1.118830  2.293438
      Bipartite    0.101878  0.019155
      Linear       0.249821  0.088568
      MLP          0.301336  0.123531
      Shapley      1.145322  2.400562
train Banzhaf      0.298161  0.121085
      BetaShapley  1.118160  2.324861
      Bipartite    0.104324  0.020459
      Linear       0.238748  0.082760
      MLP          0.307217  0.123618
      Shapley      1.144079  2.431317


Results saved to 'aggregate_results_by_dataset.csv' and 'aggregate_results_overall.csv'

Best method for each metric:
mae: Bipartite (value: 0.1019 ± 0.0383)
mse: Bipartite (value: 0.0192 ± 0.0112)
